# Testing notebook

In [1]:
import xlsxwriter
import pandas as pd
import numpy as np

from classification_power_predictor import classification_power_predictor

## We need some testing data

In [2]:
# frame size
n = 1000

test_frame = pd.DataFrame({"numeric_variable": np.random.normal(10, 5, n), 
                           "object_variable": np.round(np.random.uniform(0, 10, n)).astype('O'),
                           "column with soo000oo000oo000oo000oo long name": np.random.normal(10, 5, n)})

nv = test_frame["numeric_variable"]
ov = test_frame["object_variable"]

probs1 = nv.apply(lambda x: (x/(max(nv) - min(nv))) + ((np.random.rand())/5))
probs2 = ov.apply(lambda x: (x/(max(ov) - min(ov))) + ((np.random.rand())/5))

f_probs = (probs1 + probs2)/2
f_probs[f_probs >= 1] = 1


Y = np.zeros(n)
Y[f_probs > 0.4] = 1
Y[f_probs > 0.7] = 2

Y = pd.Series(Y)
Y_strs = Y.replace({0: 'cetegory 0', 1:'category 1', 2:'category 2'})
Y.value_counts()
Y_binary = Y.replace({2:1})

na_containts_frame = test_frame.copy()

for col in test_frame.columns:
    na_containts_frame.loc[na_containts_frame.sample(int(n/50)).index, col] = np.NaN

## Computions funcitons

In [3]:
from computions import *

### get_describe_nominal function test

Basic situation

In [4]:
get_describe_nominal(test_frame.iloc[:,1], Y)

,object_variable,1.0,1.0%,0.0,0.0%,2.0,2.0%
6.0,123,112,91.056911,7.0,5.691057,4.0,3.252033
1.0,113,22,19.469027,91.0,80.530973,0.0,0.000000
8.0,110,80,72.727273,0.0,0.000000,30.0,27.272727
2.0,107,21,19.626168,86.0,80.373832,0.0,0.000000
9.0,102,51,50.000000,0.0,0.000000,51.0,50.000000
3.0,94,48,51.063830,46.0,48.936170,0.0,0.000000
7.0,91,80,87.912088,1.0,1.098901,10.0,10.989011
5.0,87,79,90.804598,7.0,8.045977,1.0,1.149425
4.0,82,63,76.829268,19.0,23.170732,0.0,0.000000
0.0,49,4,8.163265,45.0,91.836735,0.0,0.000000


Data frame with emptys

In [5]:
test = get_describe_nominal(na_containts_frame.iloc[:,1], Y)
print("Is all obs. are in table - ",  str(test['object_variable'].sum() == n))
test = get_describe_nominal(na_containts_frame.fillna('empty').iloc[:,1], Y)
print("Is all obs. are in table - ",  str(test['object_variable'].sum() == n))

Is all obs. are in table -  False
Is all obs. are in table -  True


### AUC computing test

AUC numeric - simple fucntion

In [6]:
print(get_AUC_numeric(test_frame['numeric_variable'] ,Y))
print(get_AUC_numeric(test_frame['numeric_variable'], Y_strs))
print(get_AUC_numeric(na_containts_frame['numeric_variable'], Y_strs))
print(get_AUC_numeric(test_frame['numeric_variable'], Y_binary))

{1.0: 0.5509977149700014, 0.0: 0.30096396516062923, 2.0: 0.7585713156299685}
{'category 1': 0.5509977149700014, 'cetegory 0': 0.30096396516062923, 'category 2': 0.7585713156299685}
{'category 1': 0.548858505564388, 'cetegory 0': 0.30387624466571833, 'category 2': 0.7567294291257891}
{1.0: 0.6990360348393708, 0.0: 0.30096396516062923}


AUC nominal

In [7]:
non_table = get_AUC_nominal(test_frame['object_variable'], Y)
dn_tab = get_describe_nominal(test_frame.iloc[:,1], Y_strs)
with_table = get_AUC_nominal(test_frame['object_variable'], Y_strs, dn_tab)
print(with_table)

print('AUC bynary')
print(get_AUC_nominal(test_frame['object_variable'], Y_binary))

print('AUC with emptys')
print(get_AUC_numeric(na_containts_frame['object_variable'], Y_strs))

{'category 1': 0.8389461164672865, 'cetegory 0': 0.933013434790034, 'category 2': 0.9297253138203955}
AUC bynary
{1.0: 0.933013434790034, 0.0: 0.933013434790034}
AUC with emptys
{'category 1': 0.6749472647711694, 'cetegory 0': 0.06687855618776675, 'category 2': 0.9289815472948648}


Get publish AUC for any predictor type

In [8]:
non_table = get_full_AUC(test_frame['object_variable'], Y, 'nominal')
print('nominal predictor without table')
print(non_table)


with_table = get_full_AUC(test_frame['object_variable'], Y_strs, 'nominal', dn_tab)
print('moninal predictor with table')
print(with_table)

numeric_var = get_full_AUC(test_frame['numeric_variable'], Y, 'numeric')
print('numeric predictor')
print(numeric_var)

nominal predictor without table
{1.0: {'AUC': 0.8389461164672865, 'rel_type': 1, 'GINI': 0.6778922329345729}, 0.0: {'AUC': 0.933013434790034, 'rel_type': 1, 'GINI': 0.8660268695800679}, 2.0: {'AUC': 0.9297253138203955, 'rel_type': 1, 'GINI': 0.8594506276407909}}
moninal predictor with table
{'category 1': {'AUC': 0.8389461164672865, 'rel_type': 1, 'GINI': 0.6778922329345729}, 'cetegory 0': {'AUC': 0.933013434790034, 'rel_type': 1, 'GINI': 0.8660268695800679}, 'category 2': {'AUC': 0.9297253138203955, 'rel_type': 1, 'GINI': 0.8594506276407909}}
numeric predictor
{1.0: {'AUC': 0.5509977149700014, 'rel_type': 1, 'GINI': 0.10199542994000277}, 0.0: {'AUC': 0.6990360348393707, 'rel_type': -1, 'GINI': 0.3980720696787414}, 2.0: {'AUC': 0.7585713156299685, 'rel_type': 1, 'GINI': 0.517142631259937}}


## Creating a writer to excel

In [9]:
xl_writer = pd.ExcelWriter("test_result/test.xlsx",engine='xlsxwriter')

In [10]:
my_cpp = classification_power_predictor(test_frame, Y)
my_cpp.update_predictors()
#my_cpp.write_to_book(xl_writer)

In [11]:
predictos_data = my_cpp.get_predictors_data()

KeyError: 0

## Saving the results

In [7]:
xl_writer.close()